#### Section 1.4.4 Linear Regulator
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

#### Imports

In [19]:
from pathlib import Path
import py4spice as spi

#### Path to Ngspice and the project

In [20]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Absolute path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg")

#### Prepare other paths

In [21]:
# Create sim_results directory if doesn't exist
RESULTS_PATH = PROJ_PATH / "sim_results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

#### Define netlists

In [22]:
# Create netlists directory and files if they don't exist
NETLISTS_PATH = PROJ_PATH / "netlists"
NETLISTS_PATH.mkdir(parents=True, exist_ok=True)  # Create the directory

# Create title.cir with content only if it doesn't exist
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
if not TITLE_FILENAME.exists():
    with open(TITLE_FILENAME, "w") as f:
        f.write("* First line in netlist must be a comment")
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)

# These netlist objects will be combined and a top netlist will be created later
load = spi.Netlist(NETLISTS_PATH / "load.cir")
stimulus = spi.Netlist(NETLISTS_PATH / "stimulus.cir")
supplies = spi.Netlist(NETLISTS_PATH / "supplies.cir")
models = spi.Netlist(NETLISTS_PATH / "models.cir")

# Create "end" netlist object, but no need to write to a file
end = spi.Netlist()
end.insert_line(0, ".end")

# two files we'll create later
TOP_FILENAME = NETLISTS_PATH / "top.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

# Create dut.cir from raw_kicad.cir
raw_kicad = spi.Netlist(NETLISTS_PATH / "raw_kicad.cir")
raw_kicad.del_line_starts_with(".title")  # delete first line (title)
raw_kicad.del_line_starts_with(".end")  # delete last line (.end)
raw_kicad.del_line_starts_with(".include")  # delete first  .include line
raw_kicad.del_line_starts_with(".include")  # delete second .include line
raw_kicad.del_slash()   # delete forward slashes from node names
dut = raw_kicad
dut.write_to_file(NETLISTS_PATH / "dut.cir")  # this is now the dut (device under test)

#### Create vectors

In [23]:
VEC_ALL = spi.Vectors("all")
VEC_ALL_EXPANDED = spi.Vectors(
    "q1_base div e1#branch in out out_meas vee vee#branch vin#branch vmeas#branch ref vref#branch"
)
VEC_OUT = spi.Vectors("out")
VEC_IN_OUT = spi.Vectors("in out")
VEC_INTEREST = spi.Vectors("in out div q1_base")
VEC_POWER_CALC = spi.Vectors("in vin#branch out vmeas#branch")
VEC_ETA = spi.Vectors("eta")

#### Define analyses

In [24]:
list_of_analyses: list[spi.Analyses] = [] # create empty list. Next sections define

# 1st analysis: operating point
# for this simulation, we'll have only one analysis
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(op1)

# 2nd analysis
dc_cmd = "dc vin 6 17 0.1"
dc1 = spi.Analyses("dc1", "dc", dc_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(dc1)

#### Create control section

In [25]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME)  # create netlist object
spi.print_section("Control File", my_control)  # print out contents


--- Control File ---
.control
* Timestamp: Wed Feb 21 02:29:41 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/op1.txt
dc vin 6 17 0.1
wrdata /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/dc1.txt all
quit
.endc
--------------------



#### Simulate

In [26]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = title + dut + load + supplies + stimulus + models + control + end

top.write_to_file(TOP_FILENAME)
spi.print_section("top netlist", top)

# prepare simulate object, print out command, and simulate
sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run() # run the Ngspice simulation


--- top netlist ---
* linear regulator, section 1.4.4
rf1 beta rc 100
cf1 rc div 100n
rlower1 div com 10k
e1 beta com vref div 100
rupper1 out div 10k
e2 g com beta com 10
x2 in gain gain k=0.05
rsol1 sum out 1
x3 gain g sum sum k1=1 k2=1 a=1
rload out out_meas 100
vmeas out_meas 0 dc 0
vee vee 0 dc 0
vref vref 0 dc 2.5
vin in 0 dc 12
.subckt gain in out k=1
e1 out 0 in 0 {k}
.ends gain

.subckt sum in1 in2 out k1=1 k2=1 a=1
b1 out 0 v = {a * ((v(in1) * k1) + (v(in2) * k2))}
.ends sum
.control
* timestamp: wed feb 21 02:29:41 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/op1.txt
dc vin 6 17 0.1
wrdata /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/dc1.txt all
quit
.endc
.end
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/netlists/top.

#### Simulation results to numpy

In [27]:
sim_results = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

dup indexes are: [15]


#### Operating point values from op1 analysis

In [28]:
# 1st simulation was operating point
spi.print_section("Operating Point Results", sim_results[0].print_table())

0.0

--- Operating Point Results ---
b.x3.b1#branch  -0.0250057
beta             2.443076
com              2.500575
div              2.500575
e.x2.e1#branch   0.0
e1#branch       -3.36103e-18
e2#branch        0.0
g                1.925581
gain             0.6
in               12.0
out              2.500575
out_meas         0.0
rc               2.443076
sum              2.525581
vee              0.0
vee#branch       0.0
vin#branch       0.0
vmeas#branch     0.02500575
vref             2.5
vref#branch      0.0
-------------------------------



#### Results from DC transfer analysis

In [ ]:
# make a copy of simresults[1] (dc analysis) and reduce signals necessary for power calc
pwr1 = copy.deepcopy(sim_results[1])
pwr1.vec_subset(VEC_POWER_CALC.list_out())
pwr1.multiply("in", "vin#branch", "pin")        # calc input power
pwr1.multiply("out", "vmeas#branch", "pout")    # calc output power
pwr1.divide("pout", "pin", "eta_neg")           # calc efficiency
pwr1.scaler(-100, "eta_neg", "eta")             # make positive value & convert to %

# make a copy and reduce signals to just "eta"
pwr2 = copy.deepcopy(pwr1)
pwr2.vec_subset(VEC_ETA.list_out())

# plot the efficiency vs. Vin
plot_data = pwr2.x_axis_and_sigs(VEC_ETA.list_out())
y_names = VEC_ETA.list_out()
my_plt = spi.Plot("my_plot", plot_data, y_names, RESULTS_PATH)
my_plt.set_title("Power Efficiency")
my_plt.define_axes(("Vin", "voltage", "linear"), ("efficiency", "%", "linear"))